# 模块导入

In [ ]:
import torch
import torch.nn as nn
from timm.models.layers import trunc_normal_, DropPath, to_2tuple
import os
from model.blocks import Mlp  # 复用前面定义好的 MLP 模块

# 自定义 Query Attention 注意力模块

In [ ]:
class query_Attention(nn.Module):
    """
    全局模块专用自注意力机制，使用固定可训练的 Query 向量
    """
    def __init__(self, dim, num_heads=2, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # 固定长度可训练 Query
        self.q = nn.Parameter(torch.ones((1, 10, dim)), requires_grad=True)
        self.k = nn.Linear(dim, dim, bias=qkv_bias)
        self.v = nn.Linear(dim, dim, bias=qkv_bias)

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape

        # 计算 K 和 V
        k = self.k(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        v = self.v(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

        # 生成广播型 Q 向量
        q = self.q.expand(B, -1, -1).view(B, -1, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

        # 多头注意力计算
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, 10, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

# 全局模块的自注意力残差块

In [ ]:
class query_SABlock(nn.Module):
    """
    全局模块：自注意力 + MLP 残差模块
    """
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0.,
                 attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()

        self.pos_embed = nn.Conv2d(dim, dim, 3, padding=1, groups=dim)  # 位置卷积增强
        self.norm1 = norm_layer(dim)
        self.attn = query_Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                                     attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x + self.pos_embed(x)
        x = x.flatten(2).transpose(1, 2)
        x = self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

# 卷积特征提取模块（下采样特征抽取）

In [ ]:
class conv_embedding(nn.Module):
    """
    使用卷积提取初步全局特征（下采样两次）
    """
    def __init__(self, in_channels, out_channels):
        super(conv_embedding, self).__init__()
        self.proj = nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 2, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_channels // 2),
            nn.GELU(),
            nn.Conv2d(out_channels // 2, out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.proj(x)

# 完整的 Global Prediction 模块

In [ ]:
class Global_pred(nn.Module):
    """
    全局模块，预测颜色矩阵 + gamma 值，用于全局曝光色彩调整
    """
    def __init__(self, in_channels=3, out_channels=64, num_heads=4, type='exp'):
        super(Global_pred, self).__init__()

        # γ (gamma) 初始化：曝光增强时固定为1
        if type == 'exp':
            self.gamma_base = nn.Parameter(torch.ones((1)), requires_grad=False)
        else:
            self.gamma_base = nn.Parameter(torch.ones((1)), requires_grad=True)

        # 初始化基础颜色矩阵为单位矩阵
        self.color_base = nn.Parameter(torch.eye(3), requires_grad=True)

        # 卷积特征提取 + 注意力建模
        self.conv_large = conv_embedding(in_channels, out_channels)
        self.generator = query_SABlock(dim=out_channels, num_heads=num_heads)

        # γ 和 color 输出层
        self.gamma_linear = nn.Linear(out_channels, 1)
        self.color_linear = nn.Linear(out_channels, 1)

        self.apply(self._init_weights)

        # 注意力权重初始化：抑制 V分支，训练更稳定
        for name, p in self.named_parameters():
            if name == 'generator.attn.v.weight':
                nn.init.constant_(p, 0)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        x = self.conv_large(x)  # 卷积提取特征
        x = self.generator(x)   # 通过自注意力建模全局信息

        gamma, color = x[:, 0].unsqueeze(1), x[:, 1:]
        gamma = self.gamma_linear(gamma).squeeze(-1) + self.gamma_base
        color = self.color_linear(color).squeeze(-1).view(-1, 3, 3) + self.color_base

        return gamma, color

# 简单测试模块

In [ ]:
if __name__ == "__main__":
    os.environ['CUDA_VISIBLE_DEVICES'] = '3'
    img = torch.Tensor(8, 3, 400, 600)
    global_net = Global_pred()
    gamma, color = global_net(img)
    print(gamma.shape, color.shape)